### 1. Client erstellen

In [1]:
import os
from dotenv import load_dotenv
from edge_benchmarking_client.client import EdgeBenchmarkingClient

In [2]:
load_dotenv()

# Connection information
PROTOCOL = "https"
HOST = "api.edge-farm.agrifood-tef.edvsz.hs-osnabrueck.de"

# Basic API authentication
BASIC_AUTH_USERNAME = os.getenv("BASIC_AUTH_USERNAME")
BASIC_AUTH_PASSWORD = os.getenv("BASIC_AUTH_PASSWORD")

# Create the client
client = EdgeBenchmarkingClient(
    protocol=PROTOCOL,
    host=HOST,
    username=BASIC_AUTH_USERNAME,
    password=BASIC_AUTH_PASSWORD,
)

2024-07-04 19:30:00.862 INFO client - __init__: Created Edge Benchmarking client for user 'admin' and Edge Farm at 'https://api.edge-farm.agrifood-tef.edvsz.hs-osnabrueck.de'.
2024-07-04 19:30:00.909 INFO client - _test_connection: Edge Farm API is reachable with status code 200.


### 2. Benchmark-Daten finden und einlesen

In [3]:
EXAMPLE_ROOT_DIR = "densenet_onnx"

dataset = client.find_dataset(
    root_dir=EXAMPLE_ROOT_DIR, file_extensions={".jpg", ".jpeg"}
)
model = client.find_model(root_dir=EXAMPLE_ROOT_DIR)
model_metadata = client.find_model_metadata(root_dir=EXAMPLE_ROOT_DIR)
labels = client.find_labels(root_dir=EXAMPLE_ROOT_DIR)

2024-07-04 19:30:00.940 INFO client - find_dataset: Found dataset containing 3 samples with type(s) {'.jpeg', '.jpg'} in '/home/jovyan/work/edge-benchmarking-client/examples/densenet_onnx'.
2024-07-04 19:30:00.951 INFO client - _find_file: Found file '/home/jovyan/work/edge-benchmarking-client/examples/densenet_onnx/model/densenet_onnx.onnx' with extension {'.onnx', '.pth', '.pt'} in '/home/jovyan/work/edge-benchmarking-client/examples/densenet_onnx'.
2024-07-04 19:30:00.956 WARNING client - _find_file: Found multiple (2) files with extension {'.pbtxt'} in '/home/jovyan/work/edge-benchmarking-client/examples/densenet_onnx'. Selecting first match.
2024-07-04 19:30:00.959 INFO client - _find_file: Found file '/home/jovyan/work/edge-benchmarking-client/examples/densenet_onnx/model/config.pbtxt' with extension {'.pbtxt'} in '/home/jovyan/work/edge-benchmarking-client/examples/densenet_onnx'.
2024-07-04 19:30:00.966 WARNING client - _find_file: Found multiple (2) files with extension {'.txt

### 3. Benchmark-Job starten und auf Ergebnisse warten

In [4]:
benchmark_results, inference_results = client.benchmark(
    edge_device="edge-03",
    dataset=dataset,
    model=model,
    model_name=EXAMPLE_ROOT_DIR,
    model_metadata=model_metadata,
    labels=labels,
    num_classes=10,
    scaling="inception",
)

2024-07-04 19:30:09.089 INFO client - upload_benchmark_data: 200 - {'bucket_name': '75edf40c-29ad-49de-9e64-809b9d550b77', 'dataset': ['dataset/vulture.jpeg', 'dataset/mug.jpg', 'dataset/car.jpg'], 'labels': 'models/densenet_onnx/densenet_labels.txt', 'model': 'models/densenet_onnx/1/model.onnx', 'model_metadata': 'models/densenet_onnx/config.pbtxt', 'model_repository': 's3://https://s3.edge-farm.agrifood-tef.edvsz.hs-osnabrueck.de:443/75edf40c-29ad-49de-9e64-809b9d550b77/models'}
2024-07-04 19:30:14.282 INFO client - get_benchmark_job_results: Results for benchmark job '75edf40c-29ad-49de-9e64-809b9d550b77' are not yet available.
2024-07-04 19:30:15.340 INFO client - get_benchmark_job_results: Results for benchmark job '75edf40c-29ad-49de-9e64-809b9d550b77' are not yet available.


### 4. Ergebnisse abrufen und verarbeiten

In [5]:
import torch
import numpy as np
import pandas as pd
import torch.nn.functional as F

from collections import defaultdict

2024-07-04 19:30:19.741 INFO utils - _init_num_threads: NumExpr defaulting to 4 threads.


In [6]:
# Benchmark results
benchmark_results = pd.DataFrame(benchmark_results)
print(benchmark_results)

                         time        uptime  CPU1  CPU2  CPU3  CPU4  CPU5  \
0  2024-07-04T21:30:12.722200  P2DT7H8M7.4S    20    13     3     2     7   
1  2024-07-04T21:30:12.722842  P2DT7H8M7.4S    20    13     3     2     7   
2  2024-07-04T21:30:13.693856  P2DT7H8M8.4S     6     2     3     4     7   
3  2024-07-04T21:30:14.651403  P2DT7H8M9.4S    11     3     4     4     4   

   CPU6  CPU7  CPU8  ...  Temp SOC0  Temp SOC1  Temp SOC2  Temp Tboard  \
0     4     5     3  ...     40.593     39.937     39.781         33.0   
1     4     5     3  ...     40.593     39.937     39.781         33.0   
2     4     3     2  ...     40.531     40.125     39.906         33.0   
3     5     7     4  ...     40.437     40.062     40.156         33.0   

   Temp Tdiode  Temp tj  Power tj  Power TOT jetson_clocks nvp model  
0         34.0   43.437      2904       6992           OFF      MAXN  
1         34.0   43.437      2904       6992           OFF      MAXN  
2         34.0   43.375      2

In [7]:
# Inference results
final_inference_results = defaultdict(list)
for inference_respone_id, inference_result in inference_results.items():
    predictions = np.stack(inference_result)

    logits = predictions[:, 0].astype(float)
    probabilities = F.softmax(torch.tensor(logits), dim=0)

    predicted_classes = predictions[:, -1]
    predicted_class_index = probabilities.argmax()
    predicted_probability = probabilities.max()
    predicted_class = predicted_classes[predicted_class_index]

    final_inference_results["response id"].append(inference_respone_id)
    final_inference_results["class"].append(predicted_class)
    final_inference_results["probability"].append(
        predicted_probability.item() * 100
    )

    inference_results_df = pd.DataFrame(final_inference_results)
print(inference_results_df)

      response id       class  probability
0       0_car.jpg  SPORTS CAR    64.724496
1       1_mug.jpg  COFFEE MUG    83.965413
2  2_vulture.jpeg     VULTURE    99.069402
